In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In this competition, you'll write an algorithm to classify whether images contain either a dog or a cat.

This Competion contains 25000 Training Images of either Dog/Cat. Each image contains 224-Pixels

**Each image is having 3 Channels / Layers i.e Red-channel, Green-channel and Blue-channel. Each Pixel in each Channel each has a Pixel-intensity value between 0 & 255. These 224-Pixels are our Features and their values are Feature Values to train our Model**

This is how 'computer-sees', also called ***Computer Vision***. I also will not do much Feature Engineering as it may lead to TAMPERED Data

A colored image is typically composed of multiple colors and almost all colors can be generated from three primary colors – red, green and blue.

Hence, in the case of a colored image, there are three Matrices (or channels) – Red, Green, and Blue. Each matrix has values between 0-255 representing the intensity of the color for that pixel. Consider the below image to understand this concept:

![RGB-Image](https://cdn.analyticsvidhya.com/wp-content/uploads/2019/08/article-image-41.png)

## **Get Image Pixels**

Color images are represented as three-dimensional Numpy arrays - a collection of three two-dimensional arrays, one each for red, green, and blue channels. Each one, like grayscale arrays, has one value per pixel and their ranges are identical.

![rgb](https://e2eml.school/images/image_processing/three_d_array.png)

## **Convolution Operation**

The fundamental difference between a densely connected layer and a convolution layer is this: Dense layers learn global patterns in their input feature space (for example, for a MNIST digit, patterns involving all pixels), whereas convolution layers learn local patterns in the case of images, patterns found in small 2D windows of the inputs.

**This key characteristic gives convnets two interesting properties:**
1. The patterns they learn are translation invariant. After learning a certain pattern in the lower-right corner of a picture, a convnet can recognize it anywhere: forexample, in the upper-left corner. A densely connected network would have to learn the pattern anew if it appeared at a new location.
1. They can learn spatial hierarchies of patterns (see figure 5.2). A first convolution layer will learn small local patterns such as edges, a second convolution layer will learn larger patterns made of the features of the first layers, and so on. This allows convnets to efficiently learn increasingly complex and abstract visual concepts (because the visual world is fundamentally spatially hierarchical).

# **Import Basic Libraries**

**more to be imported as needed**

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import PIL
import PIL.Image

import tensorflow as tf

# **Load Data**

In [3]:
train_dir = "../input/dogs-vs-cats/train.zip"

import zipfile
with zipfile.ZipFile(train_dir, "r") as f:
    f.extractall('.')

In [4]:
# File Path
IMAGE_FOLDER_PATH = "../working/train"
FILE_PATH = [os.path.join(IMAGE_FOLDER_PATH,i) for i in os.listdir(IMAGE_FOLDER_PATH)]

# Labels
labels=[i for i in os.listdir(IMAGE_FOLDER_PATH)]

# Targets
targets=[i.split('.')[0] for i in labels]
y=targets

# Dataframe
data = pd.DataFrame({'PATH':FILE_PATH,'targets':targets}) 
data

## **Train-Val Split**

In [5]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(data, test_size=0.20, random_state=42)

## **Train Data**

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen=ImageDataGenerator()

train=datagen.flow_from_dataframe(dataframe=train_df,
                                     x_col="PATH",
                                     y_col="targets",
                                     target_size=(224, 224),
                                     class_mode="binary")

train

In [ ]:
train.class_indices

## **Validation Data**

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen=ImageDataGenerator()

val=datagen.flow_from_dataframe(dataframe=val_df,
                                     x_col="PATH",
                                     y_col="targets",
                                     target_size=(224, 224),
                                     class_mode="binary")

val

In [ ]:
val.class_indices

# **EDA - Further Analysis of Data**

In [ ]:
train_df.info()

In [ ]:
train_df['targets'].value_counts()

# **Visualize the Data**

## Image-Gallery Function

In [ ]:
from keras.preprocessing.image import load_img

def gallery(nrow,ncol,name):    # This function creates a grid of images from  dataset.
    
    img_df = train_df[train_df['targets'] == name]   # little data sorting
    path_df = img_df['PATH']
    
    # Gallery using Matplotlib 
    fig, ax = plt.subplots(nrow,ncol,figsize = (12,12), dpi = 100)
    axes = ax.ravel()
    
    for idx,ax  in enumerate(axes):
        ax.imshow(load_img(path_df.iloc[idx]))
    return fig.show()

In [ ]:
gallery(5,5,name='cat')

In [ ]:
gallery(5,5,name='dog')

# **Simple CNN + MaxPooling**

Importantly, a CNN takes as input tensors of shape (image_height, image_width,image_channels)

* **Conv2D-**This layer creates a convolution kernel that is convolved with the layer input to produce a tensor of outputs. When using this layer as the first layer in a model, provide the keyword argument input_shape (tuple of integers or None, does not include the sample axis), e.g. input_shape=(128, 128, 3) for 128x128 RGB pictures in data_format="channels_last". You can use None when a dimension has variable size.

Convolutions are defined by two key parameters:
1. Size of the patches extracted from the inputs—These are typically 3 × 3 or 5 × 5. Here they were 3 × 3, which is a common choice.
1. Depth of the output feature map—The number of filters computed by the convolution.

A convolution works by sliding these windows of size 3 × 3 or 5 × 5 over the 3D input feature map, stopping at every possible location, and extracting the 3D patch of surrounding features (shape (window_height, window_width, input_depth)). Each such 3D patch is then transformed (via a tensor product with the same learned weight matrix, called the convolution kernel) into a 1D vector of shape (output_depth,). All of
these vectors are then spatially reassembled into a 3D output map of shape (height,width, output_depth). Every spatial location in the output feature map corresponds to the same location in the input feature map (for example, the lower-right corner of the output contains information about the lower-right corner of the input). For instance, with 3 × 3 windows, the vector output[i, j, :] comes from the 3D patch input[i-1:i+1, j-1:j+1, :]. 

![cnn](https://miro.medium.com/max/1400/1*vkQ0hXDaQv57sALXAJquxA.jpeg)

* **MaxPool2D-**Downsamples the input along its spatial dimensions (height and width) by taking the maximum value over an input window (of size defined by pool_size) for each channel of the input. The window is shifted by strides along each dimension.

Max pooling consists of extracting windows from the input feature maps and outputting the max value of each channel. It’s conceptually similar to convolution, except that instead of transforming local patches via a learned linear transformation (the convolution kernel), they’re transformed via a hardcoded max tensor operation. A big difference from convolution is that max pooling is usually done with 2 × 2 windows and

## **Build the Model**

In [ ]:
from keras.layers import Conv2D,MaxPool2D
from keras.models import Sequential

model1 = Sequential()

model1.add(Conv2D(32,(3,3),activation='relu',input_shape=(224,224,3))),
model1.add(MaxPool2D((2,2))),

model1.add(Conv2D(64,(3,3),activation='relu')),
model1.add(MaxPool2D((2,2))),

model1.add(Conv2D(128,(3,3),activation='relu')),
model1.add(MaxPool2D((2,2))),

model1.add(Conv2D(256,(3,3),activation='relu')),
model1.add(MaxPool2D((2,2))),

model1.add(Conv2D(256,(3,3),activation='relu')),
model1.add(MaxPool2D((2,2))),

model1.summary()

# **+ Dense + Flatten**

**You can see that the output of every Conv2D and MaxPooling2D layer is a 3D tensor of shape (height, width, channels). The width and height dimensions tend to shrink. As you go deeper in the network. The number of channels is controlled by the first argument passed to the Conv2D layers (32 or 64).**

**The next step is to feed the last output tensor into a densely connected classifier network : a stack of Dense layers. These classifiers process vectors, which are 1D, whereas the current output is a
3D tensor. First we have to flatten the 3D outputs to 1D, and then add a few Dense layers on top**

* **Flatten-**Flattens the input. Does not affect the batch size.

* **Dense-**Dense implements the operation: output = activation(dot(input, kernel) + bias) where activation is the element-wise activation function passed as the activation argument, kernel is a weights matrix created by the layer, and bias is a bias vector created by the layer (only applicable if use_bias is True). These are all attributes of Dense.

In [ ]:
from keras.layers import Conv2D,MaxPool2D,Flatten,Dense
   
model1.add(Flatten()),
model1.add(Dense(1024,activation='relu')),
model1.add(Dense(256,activation='relu')),
model1.add(Dense(32,activation='relu')),
model1.add(Dense(1,activation='sigmoid')),

model1.summary()

## **Plotting**

In [ ]:
!pip install visualkeras
import visualkeras
visualkeras.layered_view(model1, legend=True) 

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model1, 'model.png',show_shapes=True)

## **Compile**

In [ ]:
model1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                metrics=tf.keras.metrics.BinaryAccuracy())

## **Train**

In [ ]:
history1=model1.fit(train,  
                   epochs=5,
                    #     verbose="auto",
                    #     callbacks=tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3) # This callback will stop the training when there is no improvement in the loss for three consecutive epochs.,
                    #     validation_split=0.0,
                            validation_data=val,
                            shuffle=True,
                    #     class_weight=None,
                    #     sample_weight=None,
                    #     initial_epoch=0,
                    #     steps_per_epoch=None,
                    #     validation_steps=None,
                    #     validation_batch_size=None,
                    #     validation_freq=1,
                    #     max_queue_size=10,
                    #     workers=1,
                    #     use_multiprocessing=False,
                    )

## **Plotting Accuracy**

In [ ]:
# summarize history for accuracy
plt.plot(history1.history['binary_accuracy'])
plt.plot(history1.history['val_binary_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

**Horrible, My friends at Kaggle could have done much better Guessing**

# **+ Padding + Strides**

**Padding-**If you want to get an output feature map with the same spatial dimensions as the input, you can use padding. Padding consists of adding an appropriate number of rows and columns on each side of the input feature map so as to make it possible to fit center convolution windows around every input tile. For a 3 × 3 window, you add one column on the right, one column on the left, one row at the top, and one row at the bottom. For a 5 × 5 window, you add two rows 


**Strides-**The description of convolution so far has assumed that the center tiles of the convolution windows are all contiguous. But the distance between two successive windows is a parameter of the convolution, called its stride, which defaults to 1. It’s possible to have strided convolutions: convolutions with a stride higher than 1

![cnn-padding-stride](https://jehyunlee.github.io/2020/11/29/Python-DL-1-conv2d/keras_conv2d_padding.gif)

## **Build Model**

In [ ]:
from keras.layers import Conv2D,MaxPool2D,Flatten,Dense
from keras.models import Sequential

model2 = Sequential()

model2.add(Conv2D(filters=32,kernel_size=3,strides=(3,3),padding='same',activation='relu',input_shape=(224,224,3))),
model2.add(MaxPool2D(pool_size=(2, 2), strides=None, padding="same")),

model2.add(Conv2D(filters=64,kernel_size=3,strides=(3,3),padding='same',activation='relu')),
model2.add(MaxPool2D(pool_size=(2, 2), strides=None, padding="same")),

model2.add(Conv2D(filters=128,kernel_size=3,strides=(3,3),padding='same',activation='relu')),
model2.add(MaxPool2D(pool_size=(2, 2), strides=None, padding="same")),

model2.add(Conv2D(filters=256,kernel_size=3,strides=(3,3),padding='same',activation='relu')),
model2.add(MaxPool2D(pool_size=(2, 2), strides=None, padding="same")),

model2.add(Conv2D(filters=256,kernel_size=3,strides=(3,3),padding='same',activation='relu')),
model2.add(MaxPool2D(pool_size=(2, 2), strides=None, padding="same")),

model2.add(Flatten()),
model2.add(Dense(units=1024,activation='relu')),
model2.add(Dense(units=256,activation='relu')),
model2.add(Dense(units=32,activation='relu')),
model2.add(Dense(units=1,activation='sigmoid')),

model2.summary()

## **Visualize**

In [ ]:
!pip install visualkeras
import visualkeras
visualkeras.layered_view(model2, legend=True)

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model1, 'model.png',show_shapes=True)

## **Compile & Train**

In [ ]:
# Model Compile
model2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                metrics=tf.keras.metrics.BinaryAccuracy())


# Train
history2=model2.fit(train,  
                   epochs=5,
                    #     verbose="auto",
                    #     callbacks=tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3) # This callback will stop the training when there is no improvement in the loss for three consecutive epochs.,
                    #     validation_split=0.0,
                    validation_data=val,
                    shuffle=True,
                    #     class_weight=None,
                    #     sample_weight=None,
                    #     initial_epoch=0,
                    #     steps_per_epoch=None,
                    #     validation_steps=None,
                    #     validation_batch_size=None,
                    #     validation_freq=1,
                    #     max_queue_size=10,
                    #     workers=1,
                    #     use_multiprocessing=False,
                    )

## **Plotting**

In [ ]:
# Comparing Accuracy
plt.plot(history2.history['val_binary_accuracy'],'b')
plt.plot(history1.history['val_binary_accuracy'],'r')
plt.title('binary_ accuracy')
plt.ylabel('val_binary_accuracy')
plt.xlabel('epoch')
plt.legend(['model1', 'model2'], loc='upper left')
plt.show()

**So little improvement that even Matplotlib cannot pick**

# **+ Data Augmentation**

Data augmentation takes the approach of generating more training data from existing training samples, by augmenting the samples via a number of random transformations that yield believable-looking images. The goal is that at training time, your model will never see the exact same picture twice. This helps expose the model to more aspects of the data and generalize better.

Image augmentation is a technique of applying different transformations to original images which results in multiple transformed copies of the same image. Each copy, however, is different from the other in certain aspects depending on the augmentation techniques you apply like shifting, rotating, flipping, etc.

Applying these small amounts of variations on the original image does not change its target class but only provides a new perspective of capturing the object in real life. And so, we use it is quite often for building deep learning models.

These image augmentation techniques not only expand the size of your dataset but also incorporate a level of variation in the dataset which allows your model to generalize better on unseen data. Also, the model becomes more robust when it is trained on new, slightly altered images.

## **Train Data**

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen=ImageDataGenerator(
#     featurewise_center=True,
#     samplewise_center=T,
#     featurewise_std_normalization=False,
#     samplewise_std_normalization=False,
#     zca_whitening=True,
#     zca_epsilon=1e-06,
#     rotation_range=90,
#     width_shift_range=0.5,
#     height_shift_range=0.5,
#     brightness_range=None,
#     shear_range=0.0,
#     zoom_range=0.0,
#     channel_shift_range=0.0,
#     fill_mode='nearest',
#     cval=0.0,
    horizontal_flip=True,
    vertical_flip=True,
    rescale=1./255,
#     preprocessing_function=None,
#     data_format=None,
#     validation_split=0.0,
#     interpolation_order=1,
#     dtype=None
)

train=datagen.flow_from_dataframe(dataframe=train_df,
                                     x_col="PATH",
                                     y_col="targets",
                                     target_size=(224, 224),
                                     class_mode="binary")

train

**The Keras ImageDataGenerator class is not an “additive” operation. It’s not taking the original data, randomly transforming it, and then returning both the original data and transformed data. Instead, the ImageDataGenerator accepts the original data, randomly transforms it, and returns only the new, transformed data.**

## **Validation Data**

In [ ]:
val=datagen.flow_from_dataframe(dataframe=val_df,
                                     x_col="PATH",
                                     y_col="targets",
                                     target_size=(224, 224),
                                     class_mode="binary")

val

## **Build Model**

In [ ]:
# Build Model

model3 = Sequential()

model3.add(Conv2D(filters=32,kernel_size=3,strides=(3,3),padding='same',activation='relu',input_shape=(224,224,3))),
model3.add(MaxPool2D(pool_size=(2, 2), strides=None, padding="same")),

model3.add(Conv2D(filters=64,kernel_size=3,strides=(3,3),padding='same',activation='relu')),
model3.add(MaxPool2D(pool_size=(2, 2), strides=None, padding="same")),

model3.add(Conv2D(filters=128,kernel_size=3,strides=(3,3),padding='same',activation='relu')),
model3.add(MaxPool2D(pool_size=(2, 2), strides=None, padding="same")),

model3.add(Flatten()),

model3.add(Dense(units=1,activation='sigmoid')),

model3.summary()

## **Visualize**

In [ ]:
!pip install visualkeras
import visualkeras
visualkeras.layered_view(model3, legend=True)

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model3, 'model.png',show_shapes=True)

## **Compile & Train**

In [ ]:
# Model Compile
model3.compile(optimizer='adam',
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                metrics=tf.keras.metrics.BinaryAccuracy())

# Train
history3=model3.fit(train,  
                   epochs=10,
                    #     verbose="auto",
                    #     callbacks=tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=3) # This callback will stop the training when there is no improvement in the loss for three consecutive epochs.,
#                         validation_split=0.2,
                    validation_data=val,
                    shuffle=True,
                    #     class_weight=None,
                    #     sample_weight=None,
                    #     initial_epoch=0,
                    #     steps_per_epoch=None,
                    #     validation_steps=None,
                    #     validation_batch_size=None,
                    #     validation_freq=1,
                    #     max_queue_size=10,
                    #     workers=1,
                    #     use_multiprocessing=False,
                    )

## **Plotting**

In [ ]:
# Comparing Accuracy
plt.plot(history3.history['binary_accuracy'],'b')
plt.plot(history3.history['val_binary_accuracy'],'b-')
# plt.plot(history1.history['val_binary_accuracy'],'b*'')
# plt.title('Model Accuracy Comparison')
# plt.xlabel('epoch')
# plt.legend(['model3', 'model2','model1'], loc='upper left')
plt.show()

**Bit better considering only 10-epochs**

# **+Regularization**
Regularizers allow you to apply penalties on layer parameters or layer activity during optimization. These penalties are summed into the loss function that the network optimizes.

Regularization penalties are applied on a per-layer basis. The exact API will depend on the layer, but many layers (e.g. Dense, Conv1D, Conv2D and Conv3D) have a unified API.

These layers expose 3 keyword arguments:

* kernel_regularizer: Regularizer to apply a penalty on the layer's kernel
* bias_regularizer: Regularizer to apply a penalty on the layer's bias
* activity_regularizer: Regularizer to apply a penalty on the layer's output

1. **Data Augmentation-**The simplest way to reduce overfitting is to increase the size of the training data. In machine learning, we were not able to increase the size of training data as the labeled data was too costly.

But, now let’s consider we are dealing with images. In this case, there are a few ways of increasing the size of the training data – rotating the image, flipping, scaling, shifting, etc. In the below image, some transformation has been done on the handwritten digits dataset.
![data-aug](https://cdn.analyticsvidhya.com/wp-content/uploads/2018/04/Screen-Shot-2018-04-04-at-12.14.45-AM.png)
This technique is known as data augmentation. This usually provides a big leap in improving the accuracy of the model. It can be considered as a mandatory trick in order to improve our predictions.

2. **Dropout-**Dropout means that during training with some probability P a neuron of the neural network gets turned off during training. Let’s look at a visual example. A simpler version of the neural network results in less complexity that can reduce overfitting. The deactivation of neurons with a certain probability P is applied at each forward propagation and weight update step.

![dropout](https://cdn.analyticsvidhya.com/wp-content/uploads/2018/04/1IrdJ5PghD9YoOyVAQ73MJw.gif)

3. **Early stopping-**Early stopping is one of the most commonly used strategies because it is very simple and quite effective. It refers to the process of stopping the training when the training error is no longer decreasing but the validation error is starting to rise.

![early-stopping](https://cdn.analyticsvidhya.com/wp-content/uploads/2018/04/Screen-Shot-2018-04-04-at-12.31.56-AM.png)

4. **Batch Normalization-**Batch normalization (BN) can also be used as a form of regularization. Batch normalization fixes the means and variances of the input by bringing the feature in the same range.

![bn](https://theaisummer.com/static/d42512016d9b99eabb69a61bb295cd50/2e9f9/normalization.png)

## **Train Data**

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen=ImageDataGenerator(
#     featurewise_center=True,
#     samplewise_center=True,
#     featurewise_std_normalization=False,
#     samplewise_std_normalization=False,
#     zca_whitening=True,
#     zca_epsilon=1e-06,
#     rotation_range=90,
#     width_shift_range=0.5,
#     height_shift_range=0.5,
#     brightness_range=None,
#     shear_range=0.0,
    zoom_range=0.5,
#     channel_shift_range=0.0,
#     fill_mode='nearest',
#     cval=0.0,
    horizontal_flip=True,
    vertical_flip=True,
    rescale=1./255,
#     preprocessing_function=None,
#     data_format=None,
#     validation_split=0.0,
#     interpolation_order=1,
#     dtype=None
)

train=datagen.flow_from_dataframe(dataframe=train_df,
                                     x_col="PATH",
                                     y_col="targets",
                                     target_size=(224, 224),
                                     class_mode="binary")

train

## **Validation Data**

In [ ]:
val=datagen.flow_from_dataframe(dataframe=val_df,
                                     x_col="PATH",
                                     y_col="targets",
                                     target_size=(224, 224),
                                     class_mode="binary")

val

## **Build Model**

In [ ]:
from keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense, Activation, BatchNormalization
from keras.models import Sequential
from tensorflow.keras import regularizers

model4 = Sequential()

model4.add(Conv2D(filters=32,kernel_size=3,strides=(3,3),padding='same',activation='relu',input_shape=(224,224,3))),
model4.add(BatchNormalization())
model4.add(MaxPool2D(pool_size=(2, 2), strides=None, padding="same")),
model4.add(Dropout(0.25))

model4.add(Conv2D(filters=64,kernel_size=3,strides=(3,3),padding='same',activation='relu')),
model4.add(MaxPool2D(pool_size=(2, 2), strides=None, padding="same")),

model4.add(Conv2D(filters=128,kernel_size=3,strides=(3,3),padding='same',activation='relu')),
model4.add(MaxPool2D(pool_size=(2, 2), strides=None, padding="same")),

# flatten output of conv
model4.add(Flatten())

# hidden layer
model4.add(Dense(512, activation='relu', kernel_regularizer=regularizers.L1L2(l1=0.001,l2=0.001))) # Added Regularization
model4.add(Dropout(0.25))

# output layer
model4.add(Dense(1, activation='sigmoid'))

model4.summary()

## **Visualize**

In [ ]:
!pip install visualkeras
import visualkeras
visualkeras.layered_view(model4, legend=True)

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model4, 'model.png',show_shapes=True)

## **Compile & Callback & Train**

In [ ]:
# Model Compile
model4.compile(optimizer='adam',
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                metrics=tf.keras.metrics.BinaryAccuracy())

# Callbacks
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
es=tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=3) # This callback will stop the training when there is no improvement in the loss for three consecutive epochs.,
lrr = ReduceLROnPlateau(monitor='val_binary_accuracy',patience=2) 
cb = [es, lrr]

# Train
history4=model4.fit(train,  
                   epochs=20,
                    #     verbose="auto",
                        callbacks=cb, 
                    #     validation_split=0.2,
                    validation_data=val,
                    shuffle=True,
                    #     class_weight=None,
                    #     sample_weight=None,
                    #     initial_epoch=0,
                    #     steps_per_epoch=None,
                    #     validation_steps=None,
                    #     validation_batch_size=None,
                    #     validation_freq=1,
                    #     max_queue_size=10,
                    #     workers=1,
                    #     use_multiprocessing=False,
                    )

## **Save Model**

In [ ]:
# Saving Model
model4.save('cat_vs_dog_model.h5')

## **Plotting**

In [ ]:
# summarize history for accuracy
plt.plot(history4.history['binary_accuracy'])
plt.plot(history4.history['val_binary_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

# **Transfer Learning**

Transfer learning is a machine learning method where we reuse a pre-trained model as the starting point for a model on a new task.

**To put it simply—a model trained on one task is repurposed on a second, related task as an optimization that allows rapid progress when modeling the second task. By applying transfer learning to a new task, one can achieve significantly higher performance than training with only a small amount of data.**

Transfer learning is so common that it is rare to train a model for an image or natural language processing-related tasks from scratch.

![tl](https://assets-global.website-files.com/5d7b77b063a9066d83e1209c/616b35e31e352613b291f4c7_P46neY1rhMd5NbTxhCbaWWAnwJwhgCZjSLp2C2-6Pzf8sBEfAxtlhnAOV_Jq_gX-zOaztDWLrtFal42V-EDr86Gcd8QYrWh4uMxZ-_-X_Pd5tOge9EkBmFr7UxrEWLMwCNZi14WK%3Ds0.jpeg)

TensorFlow Hub is an online repository of already trained TensorFlow models that you can use. These models can either be used as is, or they can be used for Transfer Learning. Transfer learning is a process where you take an existing trained model, and extend it to do additional work. This involves leaving the bulk of the model unchanged, while adding and retraining the final layers, in order to get a different set of possible outputs.

![tl2](https://miro.medium.com/max/1000/0*xNjEPIZmPvKeqss6)

**The model that we'll use is MobileNet v2 (but any model from tf2 compatible image classifier URL from tfhub.dev would work).**

## **Train Data**

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen=ImageDataGenerator(
#     featurewise_center=True,
#     samplewise_center=True,
#     featurewise_std_normalization=False,
#     samplewise_std_normalization=False,
#     zca_whitening=True,
#     zca_epsilon=1e-06,
#     rotation_range=90,
#     width_shift_range=0.5,
#     height_shift_range=0.5,
#     brightness_range=None,
#     shear_range=0.0,
    zoom_range=0.5,
#     channel_shift_range=0.0,
#     fill_mode='nearest',
#     cval=0.0,
    horizontal_flip=True,
    vertical_flip=True,
    rescale=1./255,
#     preprocessing_function=None,
#     data_format=None,
#     validation_split=0.0,
#     interpolation_order=1,
#     dtype=None
)

train=datagen.flow_from_dataframe(dataframe=train_df,
                                     x_col="PATH",
                                     y_col="targets",
                                     target_size=(224, 224),
                                     class_mode="binary")

train

## **Validation Data**

In [10]:
val=datagen.flow_from_dataframe(dataframe=val_df,
                                     x_col="PATH",
                                     y_col="targets",
                                     target_size=(224, 224),
                                     class_mode="binary")

val

## **Download MobileNet**

Download the MobileNet model and create a Keras model from it. **MobileNet** is expecting images of 224 $\times$ 224 pixels, in 3 color channels (RGB).

With transfer learning we reuse parts of an already trained model and change the final layer, or several layers, of the model, and then retrain those layers on our own dataset.

In addition to complete models, TensorFlow Hub also distributes models without the last classification layer. These can be used to easily do transfer learning.

*Note that we're calling the partial model from TensorFlow Hub (without the final classification layer) a `feature_extractor`. The reasoning for this term is that it will take the input all the way to a layer containing a number of features. So it has done the bulk of the work in identifying the content of an image, except for creating the final probability distribution. That is, it has extracted the features of the image.*

![mobilenetv2](https://ars.els-cdn.com/content/image/1-s2.0-S2210670720309070-gr4.jpg)

In [11]:
import tensorflow_hub as hub
mobilenet ="https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/2"
feature_extractor = hub.KerasLayer(mobilenet,input_shape=(224,224,3))

# Freeze the variables in the feature extractor layer, so that the training only modifies the final classifier layer.
feature_extractor.trainable = False

## **Build Model**

In [12]:
from keras.models import Sequential
from tensorflow.keras.layers import Dense

model5 = tf.keras.Sequential()

model5.add(feature_extractor),
model5.add(Dense(1, activation='sigmoid'))

model5.summary()

## **Compile & Train**

In [13]:
# Model Compile
model5.compile(optimizer='adam',
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=tf.keras.metrics.BinaryAccuracy())

# Train
history5=model5.fit(train,  
                   epochs=10,
                    #     verbose="auto",
                    #     callbacks=cb, 
                    #     validation_split=0.2,
                    validation_data=val,
                    shuffle=True,
                    #     class_weight=None,
                    #     sample_weight=None,
                    #     initial_epoch=0,
                    #     steps_per_epoch=None,
                    #     validation_steps=None,
                    #     validation_batch_size=None,
                    #     validation_freq=1,
                    #     max_queue_size=10,
                    #     workers=1,
                    #     use_multiprocessing=False,
                    )

## **Plotting**

In [14]:
# summarize history for accuracy
plt.plot(history5.history['binary_accuracy'])
plt.plot(history5.history['val_binary_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

## **Predict**

In [15]:
pred = model5.predict(val)
pred

## **Converting Probabilities of Prediction**

In [16]:
# class map => { logit : class name }
class_map = { v: k for k, v in val.class_indices.items() }

# apply a sigmoid because our model returns logits
predict = tf.nn.sigmoid(pred)
predict = tf.where(pred < 0.5, 0, 1).numpy()

predict

In [17]:
val_df['targets']=[0 if x=='cat' else 1 for x in val_df['targets']]
val_df['predict']=predict
val_df

## **Confusion Matrix**

In [18]:
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, classification_report 
fig, ax = plt.subplots(figsize = (9, 6))

cm = confusion_matrix(val_df["targets"],val_df["predict"])

disp = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = ["cat", "dog"])
disp.plot(cmap = plt.cm.Blues, ax = ax)
ax.set_title("Validation Set")
plt.show()

## **MisClassified**

In [19]:
error_df=[row['PATH'] for i,row in val_df.iterrows() if row["targets"] != row['predict']]
error_df[:10]

In [107]:
from keras.preprocessing.image import load_img

def show_error(nrow,ncol):    # This function creates a grid of images from  dataset.
    
    # Gallery using Matplotlib 
    fig, ax = plt.subplots(nrow,ncol,figsize = (12,12), dpi = 100)
    axes = ax.ravel()
    
    for idx,ax  in enumerate(axes):
        ax.imshow(load_img(error_df[idx]))
    return fig.show()

show_error(5,5)

## **Test Data-For Submission**

In [22]:
test_dir = "../input/dogs-vs-cats/test1.zip"

import zipfile
with zipfile.ZipFile(test_dir, "r") as tst:
    tst.extractall('.')

In [31]:
# File Path
IMAGE_FOLDER_PATH = "../working/test1"
FILE_PATH = [os.path.join(IMAGE_FOLDER_PATH,i) for i in os.listdir(IMAGE_FOLDER_PATH)]

# Labels
labels=[i for i in os.listdir(IMAGE_FOLDER_PATH)]

# Dataframe
test_df = pd.DataFrame({'PATH':FILE_PATH}) 
test_df

## **Processing Test Data**

In [32]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen=ImageDataGenerator(
#     featurewise_center=True,
#     samplewise_center=True,
#     featurewise_std_normalization=False,
#     samplewise_std_normalization=False,
#     zca_whitening=True,
#     zca_epsilon=1e-06,
#     rotation_range=90,
#     width_shift_range=0.5,
#     height_shift_range=0.5,
#     brightness_range=None,
#     shear_range=0.0,
    zoom_range=0.5,
#     channel_shift_range=0.0,
#     fill_mode='nearest',
#     cval=0.0,
    horizontal_flip=True,
    vertical_flip=True,
    rescale=1./255,
#     preprocessing_function=None,
#     data_format=None,
#     validation_split=0.0,
#     interpolation_order=1,
#     dtype=None
)

test=datagen.flow_from_dataframe(dataframe=test_df,
                                     x_col="PATH",
                                     y_col=None,
                                      class_mode=None,
                                     shuffle=False,
                                     target_size=(224, 224),
                                     )

test

In [34]:
idx=[int(str(x).split('.')[0]) for x in labels]
idx[:5]

In [60]:
import numpy as np
pred=model5.predict(test) 
classes=np.argmax(pred,axis=1)
len(classes)

## **Final Submission**

In [62]:
submission=pd.DataFrame({'id': idx,
                         'label': classes,
                        })
# submission
submission.to_csv('submission_new.csv', index=False)

## Suggestions:-
* Kaggle - https://www.kaggle.com/pythonkumar
* GitHub - https://github.com/KumarPython​
* Twitter - https://twitter.com/KumarPython
* LinkedIn - https://www.linkedin.com/in/kumarpython/